In [40]:
import numpy as np
import math
from gurobipy import *

alpha, pi, N, T = 1e-03, 0.8, 50, 3
m = Model("optimal")
m.ModelSense = -1 #maximize

# Create variables and put into a dictionary
xvar = {}
for j in range(T):
    xvar[j] = m.addVar(lb = 0, ub = N, obj = 1, vtype = GRB.CONTINUOUS, name = "x_"+str(j))

# Update model to integrate new variables
m.update()

# add first constraint
row1 = np.ones(T, dtype='double')
xpr = LinExpr()
for j in range(T):
    xpr += row1[j]*xvar[j]
m.addConstr(xpr == N, name="const1")

# Set objective
obj = QuadExpr()
coefficient = 1
# for i in range(T):
#     coefficient *= (1 - alpha*(xvar[i])**pi)
#     obj += coefficient*xvar[i]
m.setObjective(xvar[0]*xvar[0] + xvar[1] + xvar[2])

m.update()

m.write("optimal.lp")

m.optimize()

code = "optimal"

if m.status == GRB.status.INF_OR_UNBD:
    print('->LP infeasible or unbounded\n')
    code = "inf_or_unbd"
if m.status == GRB.status.UNBOUNDED:
    print('->LP unbounded\n')
    code = "unbd"
if m.status == GRB.status.INFEASIBLE:
    print('->LP infeasible\n')
    code = "infeas"

print('Model Status:', m.status)
print(" -> LP optimal, value = %g\n" %(m.objval))

xsol = np.zeros(T)
for j in range(T):
    print ("%s = %g" %(xvar[j].varname, xvar[j].x))
    xsol[j] = xvar[j].x

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [5e+01, 5e+01]
  RHS range        [5e+01, 5e+01]
Presolve removed 1 rows and 3 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.02 seconds
Optimal objective 2.50000000e+03
Model Status: 2
 -> LP optimal, value = 2500

x_0 = 50
x_1 = 0
x_2 = 0
